<a href="https://colab.research.google.com/github/Tverdostup1993/Python_for_ds_tasks/blob/main/%D0%A2%D0%B2%D0%B5%D1%80%D0%B4%D0%BE%D1%81%D1%82%D1%83%D0%BF_%D0%9E_%D0%86_%22HW_2_4_kNN_%D0%9A%D1%80%D0%BE%D1%81%D0%B2%D0%B0%D0%BB%D1%96%D0%B4%D0%B0%D1%86%D1%96%D1%8F_%D1%96_%D1%82%D1%8E%D0%BD%D0%B8%D0%BD%D0%B3_%D0%B3%D1%96%D0%BF%D0%B5%D1%80%D0%BF%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D1%96%D0%B2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [5]:
# prepare_churn_splits.py

import sys
import pandas as pd

try:
    from process_bank_churn import preprocess_data
except ImportError as e:
    print("❌ Не знайшов process_bank_churn.py поруч. Покладіть його поряд або додайте шлях у sys.path.")
    raise e

def main():
    in_path = "train.csv"
    df = pd.read_csv(in_path)
    print(f"✅ train.csv завантажено: {df.shape}")

    X_train, train_targets, X_val, val_targets, input_cols, scaler, encoder = preprocess_data(df, scaler_numeric=True)

    print("🔎 Розміри після препроцесингу:")
    print(f"  X_train:       {X_train.shape}")
    print(f"  train_targets: {train_targets.shape}")
    print(f"  X_val:         {X_val.shape}")
    print(f"  val_targets:   {val_targets.shape}")

    try:
        X_train.to_parquet("X_train.parquet", index=False)
        X_val.to_parquet("X_val.parquet", index=False)
        train_targets.to_frame(name=train_targets.name or "target").to_parquet("y_train.parquet", index=False)
        val_targets.to_frame(name=val_targets.name or "target").to_parquet("y_val.parquet", index=False)
        print("💾 Збережено: X_train.parquet, X_val.parquet, y_train.parquet, y_val.parquet")
    except Exception:
        X_train.to_csv("X_train.csv", index=False)
        X_val.to_csv("X_val.csv", index=False)
        train_targets.to_frame(name=train_targets.name or "target").to_csv("y_train.csv", index=False)
        val_targets.to_frame(name=val_targets.name or "target").to_csv("y_val.csv", index=False)
        print("💾 Збережено: X_train.csv, X_val.csv, y_train.csv, y_val.csv")

    return X_train, train_targets, X_val, val_targets

if __name__ == "__main__":
    X_train, train_targets, X_val, val_targets = main()


✅ train.csv завантажено: (15000, 14)
🔎 Розміри після препроцесингу:
  X_train:       (12000, 15)
  train_targets: (12000,)
  X_val:         (3000, 15)
  val_targets:   (3000,)
💾 Збережено: X_train.parquet, X_val.parquet, y_train.parquet, y_val.parquet


1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

# 1) Створюємо kNN з параметрами за замовченням
knn = KNeighborsClassifier()

# 2) Навчаємо модель
knn.fit(X_train, train_targets)

# 3) Прогнозуємо ймовірності
y_train_pred = knn.predict_proba(X_train)[:, 1]
y_val_pred = knn.predict_proba(X_val)[:, 1]

# 4) Рахуємо AUROC
train_auc = roc_auc_score(train_targets, y_train_pred)
val_auc = roc_auc_score(val_targets, y_val_pred)

print(f"AUROC на train: {train_auc:.3f}")
print(f"AUROC на val:   {val_auc:.3f}")


AUROC на train: 0.960
AUROC на val:   0.864


Різниця між train і val ~0.096 (9.6 п.п.). Це свідчить про легке переобучення (high variance): модель надто добре запам’ятала тренувальні приклади, але на нових працює гірше.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [14]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score

knn = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 31)}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

knn_gs = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    refit=True
)
knn_gs.fit(X_train, train_targets)

best_k = knn_gs.best_params_['n_neighbors']
cv_best_auc = knn_gs.best_score_
print(f"Найкращий n_neighbors: {best_k}")
print(f"CV (mean AUROC) для найкращого k: {cv_best_auc:.4f}")

knn_best = knn_gs.best_estimator_

y_train_pred = knn_best.predict_proba(X_train)[:, 1]
y_val_pred   = knn_best.predict_proba(X_val)[:, 1]

train_auc = roc_auc_score(train_targets, y_train_pred)
val_auc   = roc_auc_score(val_targets, y_val_pred)

print(f"AUROC train (best k): {train_auc:.4f}")
print(f"AUROC val   (best k): {val_auc:.4f}")

gap = train_auc - val_auc
if val_auc < 0.65 and train_auc < 0.70:
    diagnosis = "High bias (underfitting): і train, і val низькі."
elif gap > 0.10 and train_auc >= 0.80:
    diagnosis = "High variance (overfitting): train значно вищий за val."
else:
    diagnosis = "Баланс ок: різниця між train і val помірна."
print("Діагностика:", diagnosis)


prev_knn_train_auc = 0.960
prev_knn_val_auc   = 0.864
prev_tree_val_auc  = 0.923

if prev_knn_val_auc is not None:
    print(f"\nПорівняння з базовим kNN (k=5):")
    print(f"  Було  (train/val): {prev_knn_train_auc:.4f} / {prev_knn_val_auc:.4f}")
    print(f"  Стало (train/val): {train_auc:.4f} / {val_auc:.4f}")
    print(f"  Δ val AUROC: {val_auc - prev_knn_val_auc:+.4f}")

if prev_tree_val_auc is not None:
    print(f"\nПорівняння з деревом рішень (val AUROC):")
    print(f"  Дерево: {prev_tree_val_auc:.4f}")
    print(f"  kNN(best): {val_auc:.4f}")


Найкращий n_neighbors: 30
CV (mean AUROC) для найкращого k: 0.9093
AUROC train (best k): 0.9303
AUROC val   (best k): 0.9127
Діагностика: Баланс ок: різниця між train і val помірна.

Порівняння з базовим kNN (k=5):
  Було  (train/val): 0.9600 / 0.8640
  Стало (train/val): 0.9303 / 0.9127
  Δ val AUROC: +0.0487

Порівняння з деревом рішень (val AUROC):
  Дерево: 0.9230
  kNN(best): 0.9127


GridSearch дійсно покращив kNN: модель стала стабільнішою, прибралося сильне перенавчання. Decision Tree трохи краще на цих даних (0.923 vs 0.913 на валідації), але відрив невеликий.

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [16]:
import time
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score


dt = DecisionTreeClassifier(random_state=42)


param_grid = {
    "max_depth": np.arange(1, 21, 2),
    "max_leaf_nodes": np.arange(2, 11, 1),
}


cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
grid = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    scoring="roc_auc",
    cv=cv,
    n_jobs=-1,
    refit=True,
    verbose=0,
)


t0 = time.time()
grid.fit(X_train, train_targets)
elapsed = time.time() - t0


best_params = grid.best_params_
cv_best_auc = grid.best_score_
dt_best = grid.best_estimator_

print(f"⏱️ Час пошуку: {elapsed:.2f} c")
print(f"Найкращі параметри: {best_params}")
print(f"CV (mean AUROC): {cv_best_auc:.4f}")


y_tr_pred = dt_best.predict_proba(X_train)[:, 1]
y_val_pred = dt_best.predict_proba(X_val)[:, 1]

train_auc = roc_auc_score(train_targets, y_tr_pred)
val_auc   = roc_auc_score(val_targets, y_val_pred)

print(f"AUROC train (best DT): {train_auc:.4f}")
print(f"AUROC val   (best DT): {val_auc:.4f}")


gap = train_auc - val_auc
if val_auc < 0.65 and train_auc < 0.70:
    diagnosis = "High bias (underfitting)"
elif gap > 0.10 and train_auc >= 0.80:
    diagnosis = "High variance (overfitting)"
else:
    diagnosis = "Баланс ок (помірний розрив між train і val)"
print("Діагностика:", diagnosis)


manual_val_auc = 0.9127
print(f"\nПорівняння з ручним деревом: best DT val AUROC = {val_auc:.4f} vs manual = {manual_val_auc:.4f}")
print(f"Δ val AUROC: {val_auc - manual_val_auc:+.4f}")


⏱️ Час пошуку: 5.84 c
Найкращі параметри: {'max_depth': np.int64(5), 'max_leaf_nodes': np.int64(10)}
CV (mean AUROC): 0.8992
AUROC train (best DT): 0.9015
AUROC val   (best DT): 0.9002
Діагностика: Баланс ок (помірний розрив між train і val)

Порівняння з ручним деревом: best DT val AUROC = 0.9002 vs manual = 0.9127
Δ val AUROC: -0.0125


Хоча у моделі не видно перенавчання або недонавчання, а також GridSearch — більш стабільний варіант. Але в даному випадку ручний підбір дав кращу валідційну якість

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [17]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [19]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score


dt = DecisionTreeClassifier(random_state=42)

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

rs = RandomizedSearchCV(
    estimator=dt,
    param_distributions=params_dt,
    n_iter=40,
    scoring='roc_auc',
    cv=cv,
    n_jobs=-1,
    random_state=42,
    refit=True,
    verbose=0
)

t0 = time.time()
rs.fit(X_train, train_targets)
elapsed = time.time() - t0

best_params = rs.best_params_
cv_best_auc = rs.best_score_
dt_random_search_best = rs.best_estimator_

print(f"⏱️ Час пошуку: {elapsed:.2f} c")
print(f"Найкращі параметри (RandomizedSearch): {best_params}")
print(f"CV (mean AUROC): {cv_best_auc:.4f}")


y_tr_pred = dt_random_search_best.predict_proba(X_train)[:, 1]
y_val_pred = dt_random_search_best.predict_proba(X_val)[:, 1]

train_auc = roc_auc_score(train_targets, y_tr_pred)
val_auc   = roc_auc_score(val_targets, y_val_pred)

print(f"AUROC train (RS best DT): {train_auc:.4f}")
print(f"AUROC val   (RS best DT): {val_auc:.4f}")


gap = train_auc - val_auc
if val_auc < 0.65 and train_auc < 0.70:
    diagnosis = "High bias (underfitting)"
elif gap > 0.10 and train_auc >= 0.80:
    diagnosis = "High variance (overfitting)"
else:
    diagnosis = "Баланс ок (помірний розрив між train і val)"
print("Діагностика:", diagnosis)


def subset_params(estimator, keys):
    if estimator is None: return {}
    p = estimator.get_params()
    return {k: p.get(k, None) for k in keys}

try:

    keys_to_compare = ['criterion','splitter','max_depth','max_leaf_nodes','min_samples_split','min_samples_leaf','max_features']
    rs_params_view = subset_params(dt_random_search_best, keys_to_compare)
    print("\nПараметри RS best:", rs_params_view)

    if 'dt_best' in globals():
        gs_params_view = subset_params(dt_best, keys_to_compare)
        print("Параметри GS best:", gs_params_view)

except Exception as e:
    print("\n(Порівняння з GridSearch пропущено):", e)


⏱️ Час пошуку: 1.14 c
Найкращі параметри (RandomizedSearch): {'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': np.int64(14), 'max_features': None, 'max_depth': np.int64(16), 'criterion': 'entropy'}
CV (mean AUROC): 0.9103
AUROC train (RS best DT): 0.9169
AUROC val   (RS best DT): 0.9166
Діагностика: Баланс ок (помірний розрив між train і val)

Параметри RS best: {'criterion': 'entropy', 'splitter': 'best', 'max_depth': np.int64(16), 'max_leaf_nodes': np.int64(14), 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_features': None}
Параметри GS best: {'criterion': 'gini', 'splitter': 'best', 'max_depth': np.int64(5), 'max_leaf_nodes': np.int64(10), 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None}


RandomizedSearch краще за GridSearch у цьому експерименті: модель якісніша (AUROC 0.9166 vs 0.9002), збалансованіша й отримана значно швидше.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [23]:
import pandas as pd
from process_bank_churn import preprocess_data, preprocess_new_data

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

X_train, train_targets, X_val, val_targets, input_cols, scaler, encoder = preprocess_data(train_df, scaler_numeric=True)

X_test = preprocess_new_data(test_df, input_cols, scaler, encoder)

y_test_pred = dt_random_search_best.predict_proba(X_test)[:, 1]

submission = pd.DataFrame({
    "CustomerId": test_df["CustomerId"],
    "Exited": y_test_pred
})

submission.to_csv("submission.csv", index=False)
print("✅ Файл submission.csv готовий для завантаження на Kaggle!")


✅ Файл submission.csv готовий для завантаження на Kaggle!


Завантаження не відбулось, так як дедлайн пройшов